In [ ]:
! pip3 install textattack

In [ ]:
! pip install transformers[torch]

In [ ]:
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoConfig, BertForSequenceClassification

import textattack
from textattack import Attack
from textattack import datasets
from textattack import Attacker, AttackArgs
from textattack.attack_recipes import PWWSRen2019
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack.models.wrappers import HuggingFaceModelWrapper

from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapWordNet

pd.set_option('display.max_colwidth', None)

In [ ]:
! gdown "165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd"  # Arabic_stop_words.txt
! gdown "1MPHZcco5Rh8VGye91qc0bxXd0iIECD3-"  # correct_classified_test.csv
! gdown "10Umn1MBzzMOnb0l-VtcrSfMk41LhhnhF"  # arabic_offensive_lang_detection_arabert.pt (ARABert model)

In [ ]:
arabic_stop_words=[]
with open ('./Arabic_stop_words.txt',encoding='utf-8') as f :
    for word in f.readlines() :
        arabic_stop_words.append(word.split("\n")[0])

In [ ]:
PREFIX_LIST = [
    "ال",
    "و",
    "ف",
    "ب",
    "ك",
    "ل",
    "لل",
    "\u0627\u0644",
    "\u0648",
    "\u0641",
    "\u0628",
    "\u0643",
    "\u0644",
    "\u0644\u0644",
    "س",
]
SUFFIX_LIST = [
    "ه",
    "ها",
    "ك",
    "ي",
    "هما",
    "كما",
    "نا",
    "كم",
    "هم",
    "هن",
    "كن",
    "ا",
    "ان",
    "ين",
    "ون",
    "وا",
    "ات",
    "ت",
    "ن",
    "ة",
    "\u0647",
    "\u0647\u0627",
    "\u0643",
    "\u064a",
    "\u0647\u0645\u0627",
    "\u0643\u0645\u0627",
    "\u0646\u0627",
    "\u0643\u0645",
    "\u0647\u0645",
    "\u0647\u0646",
    "\u0643\u0646",
    "\u0627",
    "\u0627\u0646",
    "\u064a\u0646",
    "\u0648\u0646",
    "\u0648\u0627",
    "\u0627\u062a",
    "\u062a",
    "\u0646",
    "\u0629",
]


# the never_split list is used with the transformers library
_PREFIX_SYMBOLS = [x + "+" for x in PREFIX_LIST]
_SUFFIX_SYMBOLS = ["+" + x for x in SUFFIX_LIST]
NEVER_SPLIT_TOKENS = list(set(_PREFIX_SYMBOLS + _SUFFIX_SYMBOLS))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter",
                                          do_lower_case=False,
                                          do_basic_tokenize=True,
                                          never_split=NEVER_SPLIT_TOKENS)

In [ ]:
# loading the model "aubmindlab/bert-base-arabertv02-twitter"
model = torch.load('arabic_offensive_lang_detection_arabert.pt',map_location=torch.device('cpu') )
model.to('cpu')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def data_reformat(data):
  """
  used to convert the data from dataframe to the tuples (text, label)
  input: dataframe
  output: list of tuples (text, label)
  """
    # create a list of data tubles
    data_tuples = [(text, label) for text, label in zip(data["tweet_clean"], data["class"])]
    # return a dataset
    return datasets.Dataset(data_tuples)

In [ ]:
"""
correct_classified_test.csv: contain the correct classified test samples from test data and
                            it is used here to generate adversarial dataset for the attack
"""
# load test data
correct_classified_test = pd.read_csv('correct_classified_test.csv')
correct_classified_test = correct_classified_test.reset_index(drop=True)

attack_dataset = data_reformat(correct_classified_test)

for example in attack_dataset:
    print("label: ", example[1], " -- text: ",example[0]['text'])

label:  1  -- text:  وش ذا والله لو مو شايفات رجال حلوين بحياتهم استغفر الله قله حياء وتخلف 👎🏻👎🏻👎🏻😡😡 بايع الكليجا
label:  1  -- text:  احلي شعور لما اوسع خرقك الضيق بزبي الكبير ياخنيث 😈👠👠 عمه كسي ديوث دياثه ممحونه كلبه كلب عمتك ساديه سالب
label:  1  -- text:  بايع الكليجا ايه بعدين تعالوا قولوا انتم شهوانين 😡
label:  1  -- text:  والله عيب اللي صار بغض النظر جميل او شين وين كرامتك وحياءك يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل تقرفت من بعض الهمج اللي بالفيديو بايع الكليجا الجنادريه
label:  1  -- text:  هءلاء هم مايسمي علماء الوهابيه داءمانظرتهم للمراه دونيه وبهيمه كيف ترتقي الامه العربيه بهءلاء الشرذمه اللذين يسمون انفسهم ب علماء امحق علم هءلاء سفهاء ومن يصدقهم يكون بهيمه ✋🏻 اقترب الظهور العلني ل المهدي وسيحررنامن هذه العقول المعفنه 😷
label:  1  -- text:  ازق من كذا تحريض مافيه الله يمرض اشكالك 😷
label:  1  -- text:  ويعه 😤👊🏻👊🏻👊🏻 ياربي شقد اكررهك 😖😖
label:  1  -- text:  بايع الكليجا وصلنا لزمن نقول الله لايبلانا اعرف الحرمه من زود الحياء تستحي 

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
"""
preparing the constrains, goal function, and search method of the attack
all of these components are used for the attack recipe
"""

# Adding the arabic stopwords
transformation = WordSwapWordNet()
constraints = [RepeatModification(), StopwordModification(stopwords = arabic_stop_words)]
goal_function = UntargetedClassification(model_wrapper)
# search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
search_method = GreedyWordSwapWIR("weighted-saliency")
recipe = Attack(goal_function, constraints, transformation, search_method)
# setting the language to arabic
recipe.transformation.language = 'arb'

In [ ]:
len(attack_dataset)

710

In [ ]:
attack_args = textattack.AttackArgs(
    num_examples=len(attack_dataset) ,
    log_to_csv="log.csv",
    # checkpoint_interval=5,
    # checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = Attacker(recipe, attack_dataset, attack_args)

# start the attack
attacker.attack_dataset()

textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/IRI/log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 30 / 671 / 9 / 710: 100%|██████████| 710/710 [22:37<00:00,  1.91s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 30     |
| Number of failed attacks:     | 671    |
| Number of skipped attacks:    | 9      |
| Original accuracy:            | 98.73% |
| Accuracy under attack:        | 94.51% |
| Attack success rate:          | 4.28%  |
| Average perturbed word %:     | 9.49%  |
| Average num. words per input: | 14.95  |
| Avg num queries:              | 78.12  |
+-------------------------------+--------+


### Attack results:

In [ ]:
results = pd.read_csv("log.csv",)

In [ ]:
results["result_type"].value_counts()

Failed        671
Successful     30
Skipped         9
Name: result_type, dtype: int64

In [ ]:
results[results["result_type"] == "Successful"].tail()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
569,بايع الكليجا انا ماعلي من اللي مسوي الهاشتاق او من اللي يرتوت و و و موب جالطني الا قناه العربيه منزله هالموضوع في احد برامجه وين يبون يوصلون او وش يبون يوصلون للناس [[قطع]] الله وجيهكم 👊🏻,بايع الكليجا انا ماعلي من اللي مسوي الهاشتاق او من اللي يرتوت و و و موب جالطني الا قناه العربيه منزله هالموضوع في احد برامجه وين يبون يوصلون او وش يبون يوصلون للناس [[صامد]] الله وجيهكم 👊🏻,0.001482,0.975317,1,0,1,445,Successful
584,بايع الكليجا ليتهم حاطين [[سم]] وسط الكليجا عشان نفتك منهن 👊🏻,بايع الكليجا ليتهم حاطين [[فسد]] وسط الكليجا عشان نفتك منهن 👊🏻,0.088619,0.532302,1,0,1,38,Successful
594,يالربع هذا [[نقص]] في عقله بلا لحد يعلق ويصفقله 😡👊🏻,يالربع هذا [[صامد]] في عقله بلا لحد يعلق ويصفقله 😡👊🏻,0.003387,0.958332,1,0,1,270,Successful
680,والله [[جاب]] راس الخروفات 👎🏻,والله [[غبار]] راس الخروفات 👎🏻,0.078227,0.897382,1,0,1,127,Successful
698,الانسانيه تنتهك في الغوطه ويذبح ويقتل الاطغال والشيوخ والنساء عرض العرب مستباح في سوريا لياتي محلل سياسي فيقول المدنيون في الغوطه يمثلون ذرع بشري للجماعات الارهابيه ومثل هذه لامور تحدث في الحرب 😠 [[شيء]] يفور الدم والله قروب جزاءسطين قروب فلسطيني انقذوا الغوطه,الانسانيه تنتهك في الغوطه ويذبح ويقتل الاطغال والشيوخ والنساء عرض العرب مستباح في سوريا لياتي محلل سياسي فيقول المدنيون في الغوطه يمثلون ذرع بشري للجماعات الارهابيه ومثل هذه لامور تحدث في الحرب 😠 [[شأن]] يفور الدم والله قروب جزاءسطين قروب فلسطيني انقذوا الغوطه,0.459109,0.600656,1,0,1,300,Successful


In [ ]:
results[results["result_type"] == "Failed"].tail()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
705,بايع الكليجا اشكال البنات الخاقين عليه 🤮😂,بايع الكليجا اشكال البنات الخاقين عليه 🤮😂,0.017117,0.017117,1,1,1,6,Failed
706,نداء ل اهلنا في اليمن دوسوا علي رءوس المرتزقه الاماراتين المحتلين ب 👞👞👢,نداء ل اهلنا في اليمن دوسوا علي رءوس المرتزقه الاماراتين المحتلين ب 👞👞👢,0.006017,0.006017,1,1,1,9,Failed
707,قله حيا وين اهلهم ذولي الله لايبلانا لهالدرجه تبعين نفسك عشان واحد مادري عنك 👞 بايع الكليجا,قله حيا وين اهلهم ذولي الله لايبلانا لهالدرجه تبعين نفسك عشان واحد مادري عنك 👞 بايع الكليجا,0.001305,0.001305,1,1,1,16,Failed
708,ثم الطحلبه 🐸🐸,ثم الطحلبه 🐸🐸,0.083075,0.083075,1,1,1,2,Failed
709,متخلف حتي الحلال حرمتوه 😷,متخلف حتي الحلال حرمتوه 😷,0.006062,0.006062,1,1,1,5,Failed


In [ ]:
results[results["result_type"] == "Skipped"].tail()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
566,بلوك ياجزمه 👞,بلوك ياجزمه 👞,0.593629,0.593629,0,0,1,1,Skipped
580,الغبار جاء لنا من السعوديه 😷 لا دبرهم,الغبار جاء لنا من السعوديه 😷 لا دبرهم,0.515921,0.515921,0,0,1,1,Skipped
606,امس شاري حساب نتفليكس وقبل شوي النت انتها حظي 💩,امس شاري حساب نتفليكس وقبل شوي النت انتها حظي 💩,0.555710,0.555710,0,0,1,1,Skipped
612,بس رخامه يا رخامه 😂😂🔪🔪,بس رخامه يا رخامه 😂😂🔪🔪,0.505606,0.505606,0,0,1,1,Skipped
631,• العامل قاعد يثبت للمعزب ان الخروف 🐑 • صار صديقه حتي ما يذبحه 😂😂,• العامل قاعد يثبت للمعزب ان الخروف 🐑 • صار صديقه حتي ما يذبحه 😂😂,0.522233,0.522233,0,0,1,1,Skipped
